# Practical Data Science Tutorial
This tutorial will introduce you how to use and apply pytorch library to a specific speech classification problem. I am gonna use a speech dataset available in Kaggle, the whole process includes data preprocessing and using pytorch to build a All-CNNs model as the classifier and classify the speech in phoneme level.

## The data file
In this tutorial I use the following 2 file for training sake:
    
    train-features.npy - training features
    train-labels.npy - training labels
And the follow file for testing:

    test-features.npy - test features

train-features.npy/test-features.npy is of shape (number of utterance, 2), denoted as data. data[:,0] is feature arrays, data[:,1] bound arrays. Each row in data is related to a single utterance, each utterance is composed of multiple phonemes, and each phonemes could be variable length part of array in a single utterance. 

More specifically, data[i,0] is also an array of phonemes (i.e. each phoneme may have variable length of feature in the array. eg. data[0,j] is the fist phoneme, data[j,k] is the second phoneme and data[k:] is the third phoneme, where 0<j<k<len(data)) which related to a single utterance "i"; and data[i,1] is an array of ints denoting the starting index of each phonemes within the single utterance "i". So, the features of phoneme i in utterance j are data[j,0][data[j,1][i]:data[j,1][i+1]].

train-labels.npy is of shape (number of total phonemes,), and the "number of total phonemes" is the sum of the number of phonemes in each utterance in [train/test]-features.npy

## Pytorch library introduction
PyTorch is an open source machine learning library for Python, based on Torch, used for applications such as natural language processing. It is primarily developed by Facebook's artificial-intelligence research group, and Uber's "Pyro" software for probabilistic programming is built on it. Referred from [1].

# Now begin data preprocessing part

## First import all the require library we  gonna use

In [4]:
import numpy as np
import torch
from torch import autograd, nn
import torch.utils.data
import os
import csv
import sys

## Load raw data
Load() and load_raw() function is used for load "[train/test]-features.npy" and "train-label.npy"

In [5]:
class Load:
    def __init__(self):
        self.test_set = None
        self.train_set = None
    @property
    def test(self):
        if self.test_set is None:
            self.test_set = np.load('./test-features.npy')
        return self.test_set

    @property
    def train(self):
        if self.train_set is None:
            self.train_set = load_raw('./train')
        return self.train_set

def load_raw(name): # return a tuple
    return (
        np.load('{}-features.npy'.format(name)),
        np.load('{}-labels.npy'.format(name))
    )

## Override torch.utils.data.Dataset
torch.utils.data.Dataset is an abstract class representing a dataset. Our custom dataset should inherit Dataset and override the following methods:
__len__ so that len(dataset) returns the size of the dataset.
__getitem__ to support the indexing such that dataset[i] can be used to get ith sample
(Referred from [2])
Since torch define its own Dataset function which will also passed as a parameter to torch's DataLoader function, Dataset defines a bridge from the raw data to the Dataloader, we can change the data form in Dataset function to decide what shape or what kind of data we passed to the DataLoader. Dataloader will return an iterator of final versioned training data and labels of each mini-batch, shaped as (data, label). 

In line #8 and #9, I split the raw dataset and form them as what they belong to. And also I rewrite "__len__" and "__getitem__" function to feed what I want to my DataLoader.

In [6]:
class myDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):  # ([samples, 2], [samples]): 
                                  # samples*(
                                  # feature: ([time, frequency], [phoneme count]),
                                  # label: [phoneme count]
                                  #)
        super(myDataset, self).__init__()
        self.dataset = dataset[0][:,0]
        self.seg_arr = dataset[0][:,1]
        self.label = dataset[1]
    

    def __len__(self):
        return self.dataset.shape[0]


    def __getitem__(self, idx):
        if self.label is not None:  # Noted that for test dataset, the label is "None"
            return self.dataset[idx], self.label[idx], self.seg_arr[idx]
        return self.dataset[idx], None, self.seg_arr[idx]

# Override collate_fn() function in torch.utils.data.DataLoader (Core of Data Preprocessing)
collate_fn() is a function in torch.utils.data.DataLoader, DataLoader will call collate_fn() function like this: "batch = self.collate_fn([self.dataset[i] for i in batch_size])" in its __next__() function of __iter__() function, for every epoch, the __iter__() function will be called once and generate a interator including all batches in a single epoch; and the __next__() will be called by __iter__() for generate each batch within the epoch. Therefore, we overwrite the collate_fn() function whose job is to recieve torch.utils.data.DataSet class and generate a tuple of data for the training routine. By default, the default coallte_fn() function will return (a batch of data, a batch of label). By overriding this function, we can arbitrarily return the form of data whatever we like to see!

Now Let's talk about data preprocessing details. The "data" in line #1 is a list of tuple generated by calling our version of __getitem__() function for "batch_size" times. Here's the trick of the preprocessing part: our aim is to train a model to classify the voice data in phoneme level not in utterance level (one utterance includes multiple phonemes), but our data is arrange in utterance level (i.e. feature[i] is a utterance consist of an array of phonemes). Also, there's another ploblem here: although feature[i] is an array of phonemes, but it has different length for all i. However, to pass the batch data to GPU for model training, We have to stack our data into a rectangle which means feature[i] needs to be padded out to the same length for all i in range(len(feature)). To overcome 2 problem stated above, we do the following things:
    1. make feature of a rectangled new shape of (batch_size, frequency, maxlen_feature), where frequency is 40, the dimension for a phoneme feature, maxlen_feature is the max length of utterance of all utterance in feature;
    2. make label of a rectangled new shape of (batch_size,  maxlen_feature);
    3. create "mask" to denoted where the actually data in the new feature created in step 1, the mask shaped as (batch_size, maxlen_feature,  maxlen_feature), to use it, when done the forward pass throung the model, the prediction matrix will do some multiplication with this mask so that it will zero out the padded data in the corresponding dimension in the prediction matrix;
    4. create "mask_loss" to denote the actually data in label (as opposed to the padded ones in newly created labels in step 2);
Finally, return a tuple containing all these four newly created data as one single batch.

In [45]:
def collate_fn(data):
    '''
        Parameters: data - a list of tuple
            - feature batch_size, ([time, frequency])
            - label batch_size*([phoneme count])
            - seg_arr batch_size*([phoneme count])
        Return tupe of numpy array: 
            - feature    batch_size, frequency, maxlen_feature (frequency is 40, which is the dimension for a phoneme)
            - label    batch_size, maxlen_feature
            - mask    batch_size, maxlen_feature, maxlen_feature
            - mask_loss mask for masking loss array use batch_size * maxlen_feature
    '''
    feature, label, seg_arr = zip(*data) # all are tuple of numpy-ndarray
    batch_size = len(feature)
    frequency = feature[0].shape[1]
    maxlen_feature = 0

    for utterance in feature:
        if utterance.shape[0] > maxlen_feature:
            maxlen_feature = utterance.shape[0]

    label_new = None if label[0] is None else np.zeros((batch_size, maxlen_feature))
    feature_new = np.zeros((batch_size, frequency, maxlen_feature))
    mask = np.zeros((batch_size, maxlen_feature, maxlen_feature))
    mask_loss = np.zeros((batch_size, maxlen_feature))

    for i in range(batch_size):
        # first, update labels and mask
        cur_arr = seg_arr[i]
        cur_len = len(feature[i])
        for j in range(len(cur_arr)):
            if label_new is not None:
                label_new[i][cur_arr[j]:len(feature[i])] = label[i][j]
            over = cur_len if j == len(cur_arr)-1 else cur_arr[j+1] 
            mask[i][cur_arr[j]:over, cur_arr[j]:over]= 1
            mask_loss[i][cur_arr[j]] = 1 
        # second, update features
        feature_new[i][:,:cur_len] = feature[i].T  # use transpose to make it (frequency, maxlen_feature) 
    feature = feature_new
    label = label_new
    return feature, label, mask, mask_loss

# Now begin model building part
I use ALL-CNN-c model described in paper$^{[3]}$. The whole architecture is as the following image shows:
![alt text](ALL_CNN_C.png "ALL-CNN-C Model")
There are totally 21 layers defined in the model (i.e. only convolutional, ReLU and Dropout layer are counted as layers). I also add some batch normalization between convolutional layers whose job is to do the covariance shift to make output of each layer more uniform in some sense.
Noted that I write a class CNNModel which inherent torch.nn.Module class. In this way, I overwirte the "$\textbf{forward}$" function which will functioned as actual calculation part in the forward pass.
As to all parameters I used in layers, simply speaking, the model take input data of shape (batch_size, frequency, maxlen_feature), treate "frequency" as the "input channels" of CNN layer; with some ReLU, Dropout and Batch Normalization done, the model output the data with the shape of (batch_size, 46, maxlen_feature), where 46 is the final number of class we want to classify each phoneme into.

In [46]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.layers = nn.ModuleList([
            nn.Conv1d(in_channels=40, out_channels=192, padding=3, kernel_size=7),
            nn.Dropout(0.2, False),
            # nn.BatchNorm1d(192),
            nn.ReLU(False),
            nn.Conv1d(in_channels=192, out_channels=192, padding=3, kernel_size=7),
            nn.BatchNorm1d(192),
            nn.ReLU(False),
            nn.Conv1d(in_channels=192, out_channels=192, padding=3, kernel_size=7),
            nn.BatchNorm1d(192),
            nn.ReLU(False),


            nn.Conv1d(in_channels=192, out_channels=192, padding=3, kernel_size=7),
            nn.Dropout(0.3, False),
            # nn.BatchNorm1d(192),
            nn.ReLU(False),
            nn.Conv1d(in_channels=192, out_channels=192, padding=3, kernel_size=7),
            nn.BatchNorm1d(192),
            nn.ReLU(False),
            nn.Conv1d(in_channels=192, out_channels=192, padding=3, kernel_size=7),
            nn.BatchNorm1d(192),
            nn.ReLU(False),


            nn.Conv1d(in_channels=192, out_channels=192, padding=3, kernel_size=7),
            nn.Dropout(0.3, False),
            # nn.BatchNorm1d(192),
            nn.ReLU(False),
            nn.Conv1d(in_channels=192, out_channels=192, padding=3, kernel_size=7),
            nn.BatchNorm1d(192),
            nn.ReLU(False),
            nn.Conv1d(in_channels=192, out_channels=46, padding=3, kernel_size=7),
            nn.BatchNorm1d(46),
            nn.ReLU(False)

        ])
        self.firstrun = True

    def forward(self, input):
        h = input
        if self.firstrun:
            print("****************************************")
            print("input: {}".format(h.size()))
        for layer in self.layers:
            h = layer(h)
            if self.firstrun:
                print("{}: {}".format(layer, h.size()))
        if self.firstrun:
            print("****************************************")
        self.firstrun = False
        return h

# Training Routine

First, we initialize all parameters such as "learning rate" and "batch size";

Second, in line #35, we create the model of ALL-CNN-C defined above and apply "weights_init" for all convolutional layers in the model;

Then, define our loss function and optimizer as cross entropy loss and Adam optimizer respectively;

Next, move all parameters of the module and variables to GPU (If available) in line #45;

Next, load our DataLoader in line #51, using our overwirted version of Dataset and collate_fn() as parameters;

Then, dive into 2 for-loop, the first one defines number of epochs to run, and the second iterate through all batches within an epoch;

Then, seal all data in variable and move the data to the GPU (If available) in line #70~ #74. Noted that torch will need all data is "torch.autograd.Variable" to calculate is gradient when doing the backpropgation;

Then, do the forward pass in line , and transpose the prediction matrix for later use;

Next, use "mask" to zero out the padding in the prediction matrix in line #78 ~ #79. After masking, the output shape is (batch_size,  max_feature,  46);

Next, mask the label data, and together with the prediction, it is passed to the loss function in line ; Here is one more thing to notice, that is, because we need to do the mask to the loss in order to discard the part of loss corresponding to the padded phonemes, we need to make "reduce=False" in line #37 when defining the loss function, which, will return loss matrix corresponding to each poneme instead of a scalar of averaged loss;

Then, zero out the padding loss in line #85 and #86, we do the backpropagation in line #87 in which we will get the corresponding gradient to each variable;

Finally, we update all the parameter in our model by calling "optimizer.step()" in line #89.

In [62]:
def to_Doubletensor(numpy_array):
    # Numpy array -> Tensor
    return torch.from_numpy(numpy_array).float()


def to_Longtensor(numpy_array):
    # Numpy array -> Tensor
    return torch.from_numpy(numpy_array).type(torch.LongTensor)


def to_variable(tensor):
    # Tensor -> Variable (on GPU if possible)
    if torch.cuda.is_available():
        # Tensor -> GPU Tensor
        tensor = tensor.cuda()
    return autograd.Variable(tensor)


def weights_init(m):
    if isinstance(m, nn.Conv2d):  # apply only to convolutional layer
        nn.init.xavier_normal(m.weight)


def training_routine():

    # Define parameters
    learning_rate = 0.01
    momentum_rate = 0.7
    minibatch_size = 1
    num_epochs = 30

    raw_data = Load()
    mydataset = myDataset(raw_data.train)
    my_net = CNNModel()  # Create the network,
    my_net.apply(weights_init)

    loss_fn = torch.nn.CrossEntropyLoss(reduce=False)  # and choose the loss function / optimizer
    optim = torch.optim.SGD(
        my_net.parameters(), lr=learning_rate, momentum=momentum_rate)
#     optim = torch.optim.Adam(
#         my_net.parameters(), lr=learning_rate)
    # optim = torch.optim.Adagrad(
    #     my_net.parameters(), lr=learning_rate, weight_decay=0)

    if torch.cuda.is_available():
        # Move the network and the optimizer to the GPU
        my_net = my_net.cuda()
        loss_fn = loss_fn.cuda()

    # Define dataset
    data_loader = torch.utils.data.DataLoader(
        mydataset, batch_size=minibatch_size,
        shuffle=True, pin_memory=True,
        collate_fn=collate_fn, drop_last=True)
    '''
        data formata:
            - feature    batch_size, frequency, maxlen_feature
            - label    batch_size, maxlen_feature
            - mask    batch_size, maxlen_feature, maxlen_feature
            - mask_loss batch_size, maxlen_feature
    '''
    ep_cnt = 0
    for epoch in range(num_epochs):
        losses = []

        i = 0
 
        for (input_val, label, mask, mask_loss) in data_loader:
            # Need to transform all numpy array to tensor then to variable to flow in network
            input_val = to_variable(to_Doubletensor(input_val))
            label = to_variable(to_Longtensor(label))
            mask = to_variable(to_Doubletensor(mask))
            mask_loss = to_variable(to_Doubletensor(mask_loss))
            optim.zero_grad()  # Reset the gradients
            prediction = torch.transpose(my_net(input_val), 1,2)  # (batch_size,  maxlen_feature,  46)
            # WARNING!!! We need to keep everything in Variable to retain gradient
            mask_sum = torch.sum(mask, 2).unsqueeze(2)
            prediction_avg = torch.matmul(mask, prediction) / (
                mask_sum + (mask_sum == 0).float())  # (batch_size,  max_feature,  46)

            # Resize prediction and label to fit to loss funciton
            label = label.resize(label.size()[0] * label.size()[1], )  # (batch_size * max_feature, )
            prediction_avg = prediction_avg.resize(prediction_avg.size()[0] *
                            prediction_avg.size()[1], prediction_avg.size()[2]) # (batch_size * max_feature, 46)
            loss = loss_fn(prediction_avg, label)  # Compute losses
            mask_loss = mask_loss.resize(mask_loss.size()[0] * mask_loss.size()[1],)
            loss = torch.sum(loss * mask_loss) / torch.sum(mask_loss)
            loss.backward()  # Backpropagate the gradients
            losses.append(loss.data.cpu().numpy())
            optim.step() # Update the network
            i += 1
            if i % 5 ==0:
                print("Epoch {} Round {} Loss: {:.4f}".format(epoch, i, np.asscalar(np.mean(losses))))
            if i >= 250: return  # This is just for show use for this tutorial
        print('Saving net for Epoch{}', ep_cnt)
        torch.save(my_net, './net_Epoch_' + str(ep_cnt) + '.pkl')
        ep_cnt += 1
    return my_net.cpu()

# Testing Routine

The logic of testing routine is almost the same with the training, except the testing part do not require computing the gradient since all parameters do not need to be updated during testing. So all to do is just perform a forward pass. I use the comment to some of the line to explain why do this way or the output shape of this line. Also, followings are the points need to be noticed.
1. set "my_net.eval()" to switch from the "training mode" to "evaluation" mode in which it will turn off some function that should not work in the testing phase such as "dropout", and also change the behaviour of some function like "Batch Normalization";
2. batch size is set to 1 to predict 1 sample in a batch;
3. turn off the "shuffle" selection in DataLoader to keep the input samples in their original order;
4. the prediction is also use "mask" and "mask_loss" to zero out the padding for generalization use (i.e. In this case we do need to do the mask actually because the size of batch size is 1).

In [63]:
def testing_routine(my_net):  # net in cpu model
    if torch.cuda.is_available():
        # Move the network and the optimizer to the GPU
        my_net = my_net.cuda()
    net = net.eval()  # switch to evaluation mode to disable drop out layer
    raw_test_data = Load()
    mydataset = myDataset(raw_test_data.test)
    data_loader = torch.utils.data.DataLoader(
        mydataset,
        batch_size=1, shuffle=False,
        collate_fn=collate_fn, pin_memory=True)  # Do not shuffle

    predictions = []
    for (input_val, _, mask, mask_loss) in data_loader:
            # Need to transform all numpy array to tensor then to variable to flow in network
            input_val = to_variable(to_Doubletensor(input_val))
            mask = to_variable(to_Doubletensor(mask))
            mask_loss = to_variable(to_Longtensor(mask_loss))

            prediction = torch.transpose(my_net(input_val), 1,2)  # (batch_size, maxlen_feature, 46)
            # pooling layer, no need to cal gradient
            mask_sum = torch.sum(mask, 2).unsqueeze(2)
            prediction_avg = torch.matmul(mask, prediction) /  (mask_sum + (mask_sum == 0).float())  # (batch_size, max_feature, 46)

            prediction_avg = prediction_avg.resize(prediction_avg.size()[0]
                            * prediction_avg.size()[1], prediction_avg.size()[2]).data.cpu().numpy()  # (batch_size * max_feature, 46)
            prediction_avg = [np.argmax(p) for p in prediction_avg]  # (batch_size * max_feature, )
            predictions += prediction_avg[np.where(mask_loss.data.cpu().numpy() !=0)].tolist()

    return predictions  # (number of phonemes in the test data set,)

## Main logic

In this module, I am gonna show the training intermida result and testing output. 

Limited to the length of content, I just show a part of training process which is enough for demonstrate the whole process.

For testing part, I just jump directly to the ".csv" file that generated in advance using my trained model. Sorry for can not wait training phase to finish due to huge amout of time consuming :-)

In [64]:
def main(argv):
    # Training settings
    net = training_routine()
#     predictions, labels = testing_routine(net)
#     acc = len(np.where(prediction == labels)[0]) / len(predictions)
    with open('Prediction_Results_20.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            print('Sample #'+row['Id']+", Label:"+row['Label'])
if __name__ == '__main__':
    main(sys.argv[1:])

****************************************
input: torch.Size([1, 40, 499])
Conv1d(40, 192, kernel_size=(7,), stride=(1,), padding=(3,)): torch.Size([1, 192, 499])
Dropout(p=0.2): torch.Size([1, 192, 499])
ReLU(): torch.Size([1, 192, 499])
Conv1d(192, 192, kernel_size=(7,), stride=(1,), padding=(3,)): torch.Size([1, 192, 499])
BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True): torch.Size([1, 192, 499])
ReLU(): torch.Size([1, 192, 499])
Conv1d(192, 192, kernel_size=(7,), stride=(1,), padding=(3,)): torch.Size([1, 192, 499])
BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True): torch.Size([1, 192, 499])
ReLU(): torch.Size([1, 192, 499])
Conv1d(192, 192, kernel_size=(7,), stride=(1,), padding=(3,)): torch.Size([1, 192, 499])
Dropout(p=0.3): torch.Size([1, 192, 499])
ReLU(): torch.Size([1, 192, 499])
Conv1d(192, 192, kernel_size=(7,), stride=(1,), padding=(3,)): torch.Size([1, 192, 499])
BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True): torch.Size([1, 192, 499])
ReLU(): torch.Siz

Sample #960, Label:33
Sample #961, Label:11
Sample #962, Label:34
Sample #963, Label:40
Sample #964, Label:44
Sample #965, Label:42
Sample #966, Label:17
Sample #967, Label:27
Sample #968, Label:22
Sample #969, Label:25
Sample #970, Label:34
Sample #971, Label:37
Sample #972, Label:36
Sample #973, Label:42
Sample #974, Label:17
Sample #975, Label:14
Sample #976, Label:12
Sample #977, Label:23
Sample #978, Label:28
Sample #979, Label:44
Sample #980, Label:6
Sample #981, Label:28
Sample #982, Label:8
Sample #983, Label:9
Sample #984, Label:26
Sample #985, Label:19
Sample #986, Label:8
Sample #987, Label:28
Sample #988, Label:22
Sample #989, Label:35
Sample #990, Label:8
Sample #991, Label:28
Sample #992, Label:21
Sample #993, Label:11
Sample #994, Label:17
Sample #995, Label:0
Sample #996, Label:36
Sample #997, Label:36
Sample #998, Label:34
Sample #999, Label:37
Sample #1000, Label:33
Sample #1001, Label:9
Sample #1002, Label:29
Sample #1003, Label:20
Sample #1004, Label:17
Sample #1005

Sample #2670, Label:25
Sample #2671, Label:25
Sample #2672, Label:8
Sample #2673, Label:1
Sample #2674, Label:34
Sample #2675, Label:32
Sample #2676, Label:30
Sample #2677, Label:25
Sample #2678, Label:34
Sample #2679, Label:8
Sample #2680, Label:26
Sample #2681, Label:27
Sample #2682, Label:8
Sample #2683, Label:28
Sample #2684, Label:34
Sample #2685, Label:16
Sample #2686, Label:14
Sample #2687, Label:15
Sample #2688, Label:8
Sample #2689, Label:25
Sample #2690, Label:8
Sample #2691, Label:27
Sample #2692, Label:32
Sample #2693, Label:8
Sample #2694, Label:28
Sample #2695, Label:23
Sample #2696, Label:38
Sample #2697, Label:9
Sample #2698, Label:33
Sample #2699, Label:27
Sample #2700, Label:27
Sample #2701, Label:15
Sample #2702, Label:8
Sample #2703, Label:41
Sample #2704, Label:16
Sample #2705, Label:28
Sample #2706, Label:13
Sample #2707, Label:17
Sample #2708, Label:0
Sample #2709, Label:37
Sample #2710, Label:22
Sample #2711, Label:25
Sample #2712, Label:33
Sample #2713, Label:2

Sample #4186, Label:16
Sample #4187, Label:43
Sample #4188, Label:43
Sample #4189, Label:8
Sample #4190, Label:17
Sample #4191, Label:17
Sample #4192, Label:28
Sample #4193, Label:44
Sample #4194, Label:32
Sample #4195, Label:18
Sample #4196, Label:32
Sample #4197, Label:17
Sample #4198, Label:8
Sample #4199, Label:11
Sample #4200, Label:16
Sample #4201, Label:37
Sample #4202, Label:3
Sample #4203, Label:36
Sample #4204, Label:36
Sample #4205, Label:37
Sample #4206, Label:16
Sample #4207, Label:44
Sample #4208, Label:44
Sample #4209, Label:8
Sample #4210, Label:28
Sample #4211, Label:37
Sample #4212, Label:11
Sample #4213, Label:11
Sample #4214, Label:8
Sample #4215, Label:14
Sample #4216, Label:26
Sample #4217, Label:23
Sample #4218, Label:34
Sample #4219, Label:11
Sample #4220, Label:14
Sample #4221, Label:37
Sample #4222, Label:40
Sample #4223, Label:42
Sample #4224, Label:8
Sample #4225, Label:37
Sample #4226, Label:34
Sample #4227, Label:20
Sample #4228, Label:26
Sample #4229, Lab

Sample #5611, Label:20
Sample #5612, Label:33
Sample #5613, Label:11
Sample #5614, Label:37
Sample #5615, Label:34
Sample #5616, Label:8
Sample #5617, Label:20
Sample #5618, Label:16
Sample #5619, Label:28
Sample #5620, Label:34
Sample #5621, Label:37
Sample #5622, Label:15
Sample #5623, Label:8
Sample #5624, Label:25
Sample #5625, Label:8
Sample #5626, Label:27
Sample #5627, Label:32
Sample #5628, Label:8
Sample #5629, Label:28
Sample #5630, Label:23
Sample #5631, Label:8
Sample #5632, Label:28
Sample #5633, Label:14
Sample #5634, Label:15
Sample #5635, Label:22
Sample #5636, Label:34
Sample #5637, Label:25
Sample #5638, Label:8
Sample #5639, Label:34
Sample #5640, Label:34
Sample #5641, Label:22
Sample #5642, Label:26
Sample #5643, Label:40
Sample #5644, Label:35
Sample #5645, Label:8
Sample #5646, Label:28
Sample #5647, Label:44
Sample #5648, Label:36
Sample #5649, Label:36
Sample #5650, Label:15
Sample #5651, Label:8
Sample #5652, Label:25
Sample #5653, Label:8
Sample #5654, Label:

...

Sample #13054, Label:44
Sample #13055, Label:8
Sample #13056, Label:14
Sample #13057, Label:8
Sample #13058, Label:28
Sample #13059, Label:37
Sample #13060, Label:34
Sample #13061, Label:37
Sample #13062, Label:22
Sample #13063, Label:24
Sample #13064, Label:8
Sample #13065, Label:28
Sample #13066, Label:8
Sample #13067, Label:25
Sample #13068, Label:8
Sample #13069, Label:27
Sample #13070, Label:16
Sample #13071, Label:28
Sample #13072, Label:23
Sample #13073, Label:37
Sample #13074, Label:22
Sample #13075, Label:2
Sample #13076, Label:16
Sample #13077, Label:14
Sample #13078, Label:15
Sample #13079, Label:16
Sample #13080, Label:33
Sample #13081, Label:20
Sample #13082, Label:33
Sample #13083, Label:11
Sample #13084, Label:32
Sample #13085, Label:34
Sample #13086, Label:8
Sample #13087, Label:20
Sample #13088, Label:16
Sample #13089, Label:28
Sample #13090, Label:34
Sample #13091, Label:37
Sample #13092, Label:15
Sample #13093, Label:8
Sample #13094, Label:25
Sample #13095, Label:8
S

Sample #14654, Label:14
Sample #14655, Label:22
Sample #14656, Label:28
Sample #14657, Label:19
Sample #14658, Label:17
Sample #14659, Label:27
Sample #14660, Label:18
Sample #14661, Label:35
Sample #14662, Label:8
Sample #14663, Label:28
Sample #14664, Label:21
Sample #14665, Label:7
Sample #14666, Label:44
Sample #14667, Label:33
Sample #14668, Label:23
Sample #14669, Label:12
Sample #14670, Label:43
Sample #14671, Label:40
Sample #14672, Label:32
Sample #14673, Label:15
Sample #14674, Label:15
Sample #14675, Label:23
Sample #14676, Label:8
Sample #14677, Label:27
Sample #14678, Label:16
Sample #14679, Label:33
Sample #14680, Label:8
Sample #14681, Label:25
Sample #14682, Label:8
Sample #14683, Label:28
Sample #14684, Label:12
Sample #14685, Label:22
Sample #14686, Label:44
Sample #14687, Label:22
Sample #14688, Label:22
Sample #14689, Label:34
Sample #14690, Label:25
Sample #14691, Label:10
Sample #14692, Label:37
Sample #14693, Label:34
Sample #14694, Label:8
Sample #14695, Label:2

Sample #16095, Label:37
Sample #16096, Label:44
Sample #16097, Label:3
Sample #16098, Label:36
Sample #16099, Label:36
Sample #16100, Label:42
Sample #16101, Label:8
Sample #16102, Label:28
Sample #16103, Label:22
Sample #16104, Label:44
Sample #16105, Label:15
Sample #16106, Label:7
Sample #16107, Label:37
Sample #16108, Label:42
Sample #16109, Label:23
Sample #16110, Label:42
Sample #16111, Label:8
Sample #16112, Label:26
Sample #16113, Label:21
Sample #16114, Label:7
Sample #16115, Label:41
Sample #16116, Label:15
Sample #16117, Label:8
Sample #16118, Label:34
Sample #16119, Label:17
Sample #16120, Label:32
Sample #16121, Label:26
Sample #16122, Label:16
Sample #16123, Label:34
Sample #16124, Label:32
Sample #16125, Label:8
Sample #16126, Label:37
Sample #16127, Label:35
Sample #16128, Label:22
Sample #16129, Label:14
Sample #16130, Label:8
Sample #16131, Label:28
Sample #16132, Label:37
Sample #16133, Label:3
Sample #16134, Label:36
Sample #16135, Label:36
Sample #16136, Label:15
S

Sample #17633, Label:22
Sample #17634, Label:26
Sample #17635, Label:22
Sample #17636, Label:37
Sample #17637, Label:40
Sample #17638, Label:21
Sample #17639, Label:8
Sample #17640, Label:44
Sample #17641, Label:23
Sample #17642, Label:33
Sample #17643, Label:30
Sample #17644, Label:37
Sample #17645, Label:8
Sample #17646, Label:14
Sample #17647, Label:37
Sample #17648, Label:40
Sample #17649, Label:40
Sample #17650, Label:40
Sample #17651, Label:32
Sample #17652, Label:11
Sample #17653, Label:28
Sample #17654, Label:37
Sample #17655, Label:0
Sample #17656, Label:15
Sample #17657, Label:7
Sample #17658, Label:37
Sample #17659, Label:22
Sample #17660, Label:41
Sample #17661, Label:41
Sample #17662, Label:18
Sample #17663, Label:1
Sample #17664, Label:32
Sample #17665, Label:7
Sample #17666, Label:27
Sample #17667, Label:7
Sample #17668, Label:27
Sample #17669, Label:14
Sample #17670, Label:22
Sample #17671, Label:33
Sample #17672, Label:16
Sample #17673, Label:25
Sample #17674, Label:37

Sample #19152, Label:28
Sample #19153, Label:37
Sample #19154, Label:34
Sample #19155, Label:37
Sample #19156, Label:6
Sample #19157, Label:38
Sample #19158, Label:8
Sample #19159, Label:5
Sample #19160, Label:8
Sample #19161, Label:15
Sample #19162, Label:17
Sample #19163, Label:44
Sample #19164, Label:19
Sample #19165, Label:17
Sample #19166, Label:27
Sample #19167, Label:37
Sample #19168, Label:33
Sample #19169, Label:11
Sample #19170, Label:28
Sample #19171, Label:37
Sample #19172, Label:22
Sample #19173, Label:25
Sample #19174, Label:7
Sample #19175, Label:35
Sample #19176, Label:40
Sample #19177, Label:8
Sample #19178, Label:28
Sample #19179, Label:26
Sample #19180, Label:28
Sample #19181, Label:14
Sample #19182, Label:8
Sample #19183, Label:37
Sample #19184, Label:17
Sample #19185, Label:27
Sample #19186, Label:44
Sample #19187, Label:3
Sample #19188, Label:28
Sample #19189, Label:40
Sample #19190, Label:25
Sample #19191, Label:7
Sample #19192, Label:35
Sample #19193, Label:40
S

# Reference
[1] https://en.wikipedia.org/wiki/PyTorch

[2] http://pytorch.org/tutorials/beginner/data_loading_tutorial.html?highlight=dataloader

[3] Springenberg, Jost Tobias, et al. "Striving for simplicity: The all convolutional net." arXiv preprint arXiv:1412.6806 (2014).